## LRO Visualization
This is an example showing how to define, manipulate and visualize a spacecraft shape

In [35]:
import trimesh as tm
import numpy as np
import spiceypy as sp
import matplotlib.pyplot as plt
import matplotlib
from pyRTX.classes.Spacecraft import Spacecraft
from matplotlib.colors import to_rgba_array

The first thing we need to do is define and load a SPICE metakernel, containing all the information (attitude kernels, ephemerides,leap seconds kernels, etc) needed. 

**NOTE**: If you don't have already downloaded the LRO kernel set needed for running the examples, first run `python download_lro_kernels.py` in the `examples` folder

In [6]:
METAKR = '../example_data/LRO/metakernel_lro.tm'
sp.furnsh(METAKR)

Then we need to define the `pyRTX.Spacecraft` object. This is an object that allows to define the shape of each part of the spacecraft, its orientation (wether it's fixed or depending on a SPICE frame) and the thermo-optical properties. 
Refer to the class documentation for further details

In [7]:
obj_path = '../example_data/LRO/' # Path where the shape files (.obj) are stored


lro = Spacecraft( 
            name = 'LRO',
            base_frame = 'LRO_SC_BUS', # Name of the spacecraft body-fixed frame
            spacecraft_model = { 
            
            
            'LRO_BUS': { 
                'file' : obj_path + 'bus_rotated.obj', # .obj file of the spacecraft component
                'frame_type': 'Spice', # type of frame (can be 'Spice' or 'UD'= User Defined)
                'frame_name': 'LRO_SC_BUS', # Name of the frame
                'center': [0.0,0.0,0.0], # Origin of the component
                'diffuse': 0.1, # Diffuse reflect. coefficient
                'specular': 0.3, # Specular reflect. coefficient
            },

            'LRO_SA': {	
            'file': obj_path + 'SA_recentred.obj',
            'frame_type': 'Spice',
            'frame_name': 'LRO_SA',
            'center': [-1,-1.1, -0.1],
            'diffuse': 0,
            'specular': 0.3,
            },


            'LRO_HGA': { 
            'file': obj_path + 'HGA_recentred.obj',
            'frame_type': 'Spice',
            'frame_name': 'LRO_HGA',
            'center':[-0.99,    -0.3,  -3.1],
            'diffuse': 0.2,
            'specular': 0.1,
            },


            }
                )

unable to load materials from: bus_rotated.mtl
specified material (foil_gold.003)  not loaded!
specified material (tex_03.003)  not loaded!
unable to load materials from: SA_recentred.mtl
specified material (tex_01.010)  not loaded!
specified material (shiny_panel.006)  not loaded!
unable to load materials from: HGA_recentred.mtl
concatenating texture: may result in visual artifacts
concatenating texture: may result in visual artifacts
concatenating texture: may result in visual artifacts
concatenating texture: may result in visual artifacts
concatenating texture: may result in visual artifacts


The warnings are due to `trimesh` library trying to infer the materials directly from the `.obj` files. We can ignore these warning as the material properties are all user-defined (through the thermo-optical coefficients defined in the `Spacecraft` instance)

We want now to visualize the `Spacecraft` to be sure that it looks how it is supposed to look.
Since LRO has moving appendages, the epoch at which we want to visualize the spacecraft matters. Let's define a first epoch:

In [12]:
epc = "2010 may 10 08:25:00" # Epoch defined as a string
epc_et0 =  sp.str2et( epc ) # Converted in J2000 seconds through SPICE

Here is a very simple way of visualizing the spacecraft:

In [107]:
mesh_color = 'red'
mesh = lro.dump(epc_et0) # Dump the spacecraft mesh at the requested epoch
mesh.unmerge_vertices()
mesh.visual.face_colors = to_rgba_array(mesh_color)*255
cam = tm.scene.Camera(fov = [30,30])


from trimesh.transformations import euler_matrix
alpha = 0*np.pi/180
beta = -50*np.pi/180
gamma = -10*np.pi/180
Re = euler_matrix(alpha, beta, gamma, 'rxyx')


transform = cam.look_at(
        [[0,0,0]], # Look towards the origin of the frame
        distance = 0.02,
    rotation = Re,
    
       
)

scene = tm.Scene([mesh], camera = cam, camera_transform = transform) # Setup the trimesh.Scene object
scene.show() # Display the scene

concatenating texture: may result in visual artifacts
concatenating texture: may result in visual artifacts
concatenating texture: may result in visual artifacts
concatenating texture: may result in visual artifacts
concatenating texture: may result in visual artifacts
concatenating texture: may result in visual artifacts
concatenating texture: may result in visual artifacts


In [ ]:
# This code block shows a generic way of plotting arrays in trimesh
# This can be used to represent forces, directions, etc. 


xaxis = np.array([1,0,0])
yaxis = np.array([0,1,0])
zaxis = np.array([0,0,1])
origin = np.array([0,0,0])
xaxis = tm.load_path(np.hstack(( origin, origin + xaxis*0.01)).reshape(-1, 2, 3))
yaxis = tm.load_path(np.hstack(( origin, origin + yaxis*0.01)).reshape(-1, 2, 3))
zaxis = tm.load_path(np.hstack(( origin, origin + zaxis*0.01)).reshape(-1, 2, 3))

xaxis.colors = np.full((1,4),matplotlib.colors.to_rgba_array('red')*255)
yaxis.colors = np.full((1,4),matplotlib.colors.to_rgba_array('green')*255)
zaxis.colors = np.full((1,4),matplotlib.colors.to_rgba_array('blue')*255)

scene = tm.Scene([mesh, xaxis, yaxis, zaxis])